# Hybrid Approach
- Deep learning to localize the sign using YOLO
- classical ML to recognize the traffic sign using SVM

## Imports

In [1]:
import os
import shutil
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import joblib
import random

c:\Users\OSAMA\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Paths

In [2]:
# YOLO dataset paths
orginal_yolo_dataset_root = "YOLODataset"
structured_yolo_dataset_root = "YOLO_dataset_structured"

# SVM dataset paths
original_svm_dataset_root = "svm_dataset/train"

# models
yolo_model_path = "saved_models/yolo_best.pt"
svm_model_path = "saved_models/svm_model.pkl"

# Dataset Manipulation

### dataset to YOLO format

In [ ]:
def copy_split_files(list_file, source_folder, img_output, lbl_output):
    """
    Service Name      : copy_split_files
    Sync/Async        : Synchronous
    Reentrancy        : Reentrant
    Parameters (in)   : list_file (str) - Path to .txt file containing relative image paths.
                        source_folder (str) - Directory containing the original images and label files.
                        img_output (str) - Destination directory for image files.
                        lbl_output (str) - Destination directory for label files.
    Parameters (inout): None
    Parameters (out)  : None
    Return value      : None
    Description       : Reads a list of image filenames and copies both the image and its corresponding label
                        (if available) to the specified destination folders.
    """

    with open(list_file, "r") as file:
        image_list = file.readlines()

    for entry in image_list:
        entry = entry.strip()
        filename = os.path.basename(entry)
        name_without_ext = os.path.splitext(filename)[0]
        label_filename = f"{name_without_ext}.txt"

        src_img_path = os.path.join(source_folder, filename)
        src_lbl_path = os.path.join(source_folder, label_filename)

        shutil.copy(src_img_path, os.path.join(img_output, filename))
        shutil.copy(src_lbl_path, os.path.join(lbl_output, label_filename))

base_yolo_path = orginal_yolo_dataset_root
source_data_dir = os.path.join(base_yolo_path, "ts")
train_file_list = os.path.join(base_yolo_path, "train.txt")
val_file_list = os.path.join(base_yolo_path, "test.txt")

output_root = structured_yolo_dataset_root
train_img_dir = os.path.join(output_root, "images/train")
val_img_dir = os.path.join(output_root, "images/val")
train_lbl_dir = os.path.join(output_root, "labels/train")
val_lbl_dir = os.path.join(output_root, "labels/val")

for path in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(path, exist_ok=True)

copy_split_files(train_file_list, source_data_dir, train_img_dir, train_lbl_dir)
copy_split_files(val_file_list, source_data_dir, val_img_dir, val_lbl_dir)

yaml_output_path = os.path.join(output_root, "data.yaml")
with open(yaml_output_path, "w") as yaml_file:
    yaml_file.write(
        f"train: {os.path.abspath(train_img_dir)}\n"
        f"val: {os.path.abspath(val_img_dir)}\n\n"
        f"nc: 4\n"
        f"names: ['prohibitory', 'danger', 'mandatory', 'other']\n"
    )

print("YOLO dataset prepared at:", output_root)

YOLO dataset prepared at: YOLO_dataset_structured


# Train YOLO detector

In [ ]:
model = YOLO("yolov8n.yaml").load("yolov8n.pt")  # build from YAML and transfer weights


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [ ]:
model.train(data="structured_yolo_dataset_root/data.yaml", epochs=5, imgsz=960, batch=16)

New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=YOLO_dataset/data.yaml, epochs=5, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, vis

In [ ]:
model = YOLO("runs/detect/train8/weights/best.pt")
model.train(data="structured_yolo_dataset_root/data.yaml", epochs=2, imgsz=960, visualize=True, batch=16, degrees=5, translate=0.1, scale=0.5, shear=2.0, flipud=0.5, fliplr=0.5, mosaic=1.0, mixup=0.1)

New https://pypi.org/project/ultralytics/8.3.133 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=runs/detect/train8/weights/best.pt, data=YOLO_dataset/data.yaml, epochs=2, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
model = YOLO("runs/detect/train9/weights/best.pt")
model.train(data="car_yolo_filtered/data.yaml", epochs=2, imgsz=960,visualize=True , batch=16)

New https://pypi.org/project/ultralytics/8.3.143 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=runs/detect/train17/weights/best.pt, data=car_yolo_filtered/data.yaml, epochs=2, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_st

In [ ]:
model = YOLO("runs/detect/train14/weights/best.pt")
model.train(data="car_yolo_filtered/data.yaml", epochs=2, imgsz=960,visualize=True , batch=16)

New https://pypi.org/project/ultralytics/8.3.143 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145  Python-3.7.9 torch-1.13.1+cpu CPU (13th Gen Intel Core(TM) i7-13620H)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=detect, mode=train, model=runs/detect/train19/weights/best.pt, data=car_yolo_filtered/data.yaml, epochs=2, patience=50, batch=16, imgsz=960, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_st

#### Predict YOLO

In [3]:
# Define the mapping of class indices to names
category_labels = ['prohibitory', 'danger', 'mandatory', 'other', 'Traffic light']

# Initialize the YOLO object detector with a pre-trained model
detector = YOLO(yolo_model_path)

# Video input file
video_source = "./CarlaVideos/traffic signs test.mp4"
capture = cv2.VideoCapture(video_source)

if not capture.isOpened():
    print("Error: Unable to open video.")
    exit()

# Frame loop
while True:
    success, frame = capture.read()
    if not success:
        break

    # Resize to match model training dimensions or display preferences
    frame = cv2.resize(frame, (1200, 960))

    # Run YOLO detection
    detections = detector.predict(source=frame, conf=0.6, verbose=False)
    prediction_boxes = detections[0].boxes

    for box in prediction_boxes:
        x_min, y_min, x_max, y_max = map(int, box.xyxy[0])
        class_index = int(box.cls[0])
        confidence = float(box.conf[0])
        class_label = f"{category_labels[class_index]} {confidence:.2f}"

        print(f"Detected: {class_label} at [{x_min}, {y_min}, {x_max}, {y_max}]")

        # Annotate the image with bounding box and label
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        cv2.putText(frame, class_label, (x_min, y_min - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display the annotated frame
    cv2.imshow("Traffic Detection", frame)

    # Exit loop if user presses 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
capture.release()
cv2.destroyAllWindows()


Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 395, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 802, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 803, 475]
Detected: prohibitory 0.96 at [751, 394, 802, 475]
Detected: prohibitory 0.96 at [

# SVM

### Load images

In [ ]:
def load_images_with_labels(data_dir, target_size=(64, 64)):
    """
    Service Name      : load_images_with_labels
    Sync/Async        : Synchronous
    Reentrancy        : Reentrant
    Parameters (in)   : data_dir (str) - Root path containing class-named subfolders.
                        target_size (tuple) - Desired image dimensions as (width, height).
    Parameters (inout): None
    Parameters (out)  : None
    Return value      : images (list) - Loaded and resized image arrays.
                        labels (list) - Corresponding integer labels.
    Description       : Loads and resizes images from directory, assigning labels based on folder names.
    """
    images = []
    labels = []

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)

        if not os.path.isdir(folder_path):
            continue

        label = int(folder_name)

        for file_name in os.listdir(folder_path):
            if not file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            file_path = os.path.join(folder_path, file_name)
            img = cv2.imread(file_path)

            if img is None:
                continue

            resized_img = cv2.resize(img, target_size)

            images.append(resized_img)
            labels.append(label)

    return images, labels

#### Preprocessing & Features Extraction

In [ ]:
def preprocess_and_extract_features(image_list, label_list, apply_augmentation=True):
    """
    Service Name      : preprocess_and_extract_features
    Sync/Async        : Synchronous
    Reentrancy        : Reentrant
    Parameters (in)   : image_list (list) - List of input images (as NumPy arrays)
                        label_list (list) - Corresponding labels for each image
                        apply_augmentation (bool) - Whether to apply simple image augmentations
    Parameters (inout): None
    Parameters (out)  : None
    Return value      : X_features (ndarray) - Array of HOG feature vectors
                        y_labels (ndarray) - Array of corresponding labels
    Description       : Preprocesses a list of images and extracts HOG features with optional augmentation.
    """
    feature_vectors = []
    output_labels = []

    for idx in range(len(image_list)):
        image = image_list[idx]
        label = label_list[idx]

        resized = cv2.resize(image, (64, 64))
        grayscale = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        normalized = grayscale.astype(np.float32) / 255.0
        variations = [normalized]

        if apply_augmentation:
            blurred = cv2.GaussianBlur(normalized, (5, 5), 0)
            variations.append(blurred)

            angle = random.choice([-10, 10])
            center = (normalized.shape[1] // 2, normalized.shape[0] // 2)
            rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv2.warpAffine(normalized, rot_matrix, (normalized.shape[1], normalized.shape[0]))
            variations.append(rotated)

        for version in variations:
            features = hog(
                version,
                pixels_per_cell=(8, 8),
                cells_per_block=(2, 2),
                feature_vector=True
            )
            feature_vectors.append(features)
            output_labels.append(label)

    return np.array(feature_vectors), np.array(output_labels)

#### Train SVM

In [ ]:
# Load raw images
images, labels = load_images_with_labels(original_svm_dataset_root)

# Extract features (with preprocessing and augmentation)
X, y = preprocess_and_extract_features(images, labels, apply_augmentation=True)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))

# Save the trained model
joblib.dump(clf, "svm_model_augmented.pkl")

              precision    recall  f1-score   support

           0       0.93      0.94      0.94       677
           1       0.89      0.97      0.93       691
           2       1.00      0.73      0.84        11
           3       0.99      0.95      0.97       149
           4       0.94      0.90      0.92        68
           5       0.97      0.95      0.96       155
           6       0.97      0.96      0.96       170
           7       0.97      0.97      0.97       365
           8       1.00      0.99      1.00       134
           9       0.95      0.91      0.93       160
          10       0.97      0.94      0.96       305
          11       0.98      0.93      0.95       210
          12       0.94      0.90      0.92       191
          13       0.96      0.89      0.93       215
          14       1.00      1.00      1.00       784
          15       1.00      1.00      1.00       385
          16       1.00      1.00      1.00       255
          17       1.00    

['svm_model_augmented.pkl']

# Final Prediction YOLO + SVM

In [5]:
# Load YOLO model
yolo_model = YOLO(yolo_model_path)

# Load trained SVM model
svm_model = joblib.load(svm_model_path)

CLASS_NAMES = [
    'Green Light', 'Red Light', 'Speed Limit 10', 'Speed Limit 100', 'Speed Limit 110',
    'Speed Limit 120', 'Speed Limit 20', 'Speed Limit 30', 'Speed Limit 40', 'Speed Limit 50',
    'Speed Limit 60', 'Speed Limit 70', 'Speed Limit 80', 'Speed Limit 90', 'Stop',
    'No vehicles', 'Veh > 3.5 tons prohibited', 'No entry', 'General caution',
    'Dangerous curve left', 'Dangerous curve right', 'Double curve', 'Bumpy road',
    'Slippery road', 'Road narrows on the right', 'Road work', 'Traffic signals',
    'Pedestrians', 'Children crossing', 'Bicycles crossing', 'Beware of ice/snow',
    'Wild animals crossing', 'End speed + passing limits', 'Turn right ahead', 'Turn left ahead',
    'Ahead only', 'Go straight or right', 'Go straight or left', 'Keep right',
    'Keep left', 'Roundabout mandatory', 'End of no passing', 'End no passing veh > 3.5 tons'
]

# Open video
cap = cv2.VideoCapture("./CarlaVideos/traffic signs test.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO on the frame
    frame = cv2.resize(frame, (1200, 960))
    height, width = frame.shape[:2]

    results = yolo_model.predict(frame, conf=0.5)
    boxes = results[0].boxes

    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped = frame[y1:y2, x1:x2]

        if cropped.size == 0 or x2 - x1 < 10 or y2 - y1 < 10:
            continue

        # Resize to 64x64
        resized = cv2.resize(cropped, (64, 64))

        # --- HOG Feature ---
        gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
        hog_feat = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

        # reshape features
        features = (hog_feat).reshape(1, -1)

        # Predict with SVM
        prediction = svm_model.predict(features)[0]
        confidence = max(svm_model.predict_proba(features)[0])

        if confidence < 0.8:
            continue

        label = f"{CLASS_NAMES[prediction]} ({confidence:.2f})"

        # Draw prediction
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Display result
    cv2.imshow("YOLO + SVM", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

c:\Users\OSAMA\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator SVC from version 1.5.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,

0: 768x960 1 prohibitory, 252.8ms
Speed: 14.0ms preprocess, 252.8ms inference, 0.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 prohibitory, 278.5ms
Speed: 8.1ms preprocess, 278.5ms inference, 0.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 prohibitory, 249.6ms
Speed: 7.6ms preprocess, 249.6ms inference, 3.6ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 prohibitory, 250.9ms
Speed: 9.1ms preprocess, 250.9ms inference, 0.0ms postprocess per image at shape (1, 3, 768, 960)

0: 768x960 1 prohibitory, 239.1ms
Speed: 12.5ms preprocess, 2

KeyboardInterrupt: 